In [2]:
from os.path import abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .getOrCreate()



Setting spark.hadoop.yarn.resourcemanager.principal to user050
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/22 12:43:33 WARN Utils: Service 'SparkUI' could not bind on port 20049. Attempting port 20050.
24/01/22 12:43:33 WARN Utils: Service 'SparkUI' could not bind on port 20050. Attempting port 20051.
24/01/22 12:43:33 WARN Utils: Service 'SparkUI' could not bind on port 20051. Attempting port 20052.
24/01/22 12:43:33 WARN Utils: Service 'SparkUI' could not bind on port 20052. Attempting port 20053.
24/01/22 12:43:33 WARN Utils: Service 'SparkUI' could not bind on port 20053. Attempting port 20054.
24/01/22 12:43:39 WARN HiveServer2CredentialProvider: Failed to get HS2 delegation token
java.util.NoSuchElementException: spark.sql.hive.hiveserver2.jdbc.url
	at org.apache.spark.SparkConf.$anonfun$get$1(SparkConf.scala:2

In [3]:
empleados=spark.read.json("files/empleados.json")
empleados.show()

24/01/22 12:44:10 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-2/pods. Message: pods "cdsw-g0iall5p0jvpi0ts-exec-1" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-g0iall5p0jvpi0ts-exec-1, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-g0iall5p0jvpi0ts-exec-1" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.assertResponseCode(OperationSupport.java:620)
	at io.fabric8.kubernetes.client.ds

+---+----+-------+
|age|  id|   name|
+---+----+-------+
| 25|1201| satish|
| 28|1202|krishna|
| 39|1203|  amith|
| 23|1204|  javed|
| 23|1205| prudvi|
+---+----+-------+



In [4]:
!rm -rf /tmp/empleados_csv
empleados.write.option("header",True).csv('/tmp/empleados_csv')

In [5]:
empleadoscsv=spark.read.options(header='True',inferSchema='True',delimiter=',')\
    .csv("files/empleados.csv")
empleadoscsv.show()

+---+----+-------+
|age|  id|   name|
+---+----+-------+
| 25|1201| satish|
| 28|1202|krishna|
| 39|1203|  amith|
| 23|1204|  javed|
| 23|1205| prudvi|
+---+----+-------+



In [6]:
data = [[295, "South Bend", "Indiana", "IN", 101190, 112.9],\
        [132, "Callisto", "Indiana", "IN", 8790, 103.9]]
columns = ["rank", "city", "state", "code", "population", "price"]
schema="rank LONG, city STRING, state STRING, code STRING, population LONG, price DOUBLE"
df1 = spark.createDataFrame(data, schema=schema)
df1.show()

+----+----------+-------+----+----------+-----+
|rank|      city|  state|code|population|price|
+----+----------+-------+----+----------+-----+
| 295|South Bend|Indiana|  IN|    101190|112.9|
| 132|  Callisto|Indiana|  IN|      8790|103.9|
+----+----------+-------+----+----------+-----+



In [7]:
empleados_select = empleados.select("age","name")
empleados_select.show()

+---+-------+
|age|   name|
+---+-------+
| 25| satish|
| 28|krishna|
| 39|  amith|
| 23|  javed|
| 23| prudvi|
+---+-------+



In [8]:
empleados_where = empleados.where(empleados["age"] > 23)
empleados_where.show()

+---+----+-------+
|age|  id|   name|
+---+----+-------+
| 25|1201| satish|
| 28|1202|krishna|
| 39|1203|  amith|
+---+----+-------+



In [9]:
!rm -rf /tmp/empleados_json
!rm -rf /tmp/empleados_csv
!rm -rf /tmp/empleados_avro
empleados_where.write\
    .format("json").save("/tmp/empleados_json")

empleados_where.write\
    .format("csv").save("/tmp/empleados_csv")

empleados_where.write\
    .format("avro").save("/tmp/empleados_avro")

In [10]:
#Save directly to HIVE
#empleados.write.mode("overwrite").saveAsTable("empleados")

24/01/22 12:46:52 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 2acdaa6e-3567-4943-8727-2de1bd24821e
24/01/22 12:46:55 ERROR DefaultRequestExecutor: Error executing request: org.apache.knox.gateway.shell.ErrorResponse: https://sonaemc-dl-idbroker0.sonaemc.z20f-vg26.cloudera.site:8444/gateway/aws-cab/cab/api/v1/credentials: HTTP/1.1 403 Forbidden
24/01/22 12:46:55 ERROR AbstractIDBClient: Cloud Access Broker response: {
  "error": "There is no mapped role for the group(s) associated with the authenticated user.",
  "auth_id": "user050"
}

24/01/22 12:46:55 WARN IDBDelegationTokenBinding: Failed to fetch credentials: There is no mapped role for the group(s) associated with the authenticated user. (user: user050)


Py4JJavaError: An error occurred while calling o102.saveAsTable.
: java.nio.file.AccessDeniedException: s3a://sonaemc/my-data/warehouse/tablespace/external/hive/empleados: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: There is no mapped role for the group(s) associated with the authenticated user. (user: user050)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:215)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3784)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3673)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$exists$34(S3AFileSystem.java:4680)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:526)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDuration(IOStatisticsBinding.java:445)
	at org.apache.hadoop.fs.s3a.S3AFileSys

In [11]:
from pyspark.sql.functions import lit
empleados2 = empleados.withColumn("country",lit("Portugal"))
empleados2.show()

+---+----+-------+--------+
|age|  id|   name| country|
+---+----+-------+--------+
| 25|1201| satish|Portugal|
| 28|1202|krishna|Portugal|
| 39|1203|  amith|Portugal|
| 23|1204|  javed|Portugal|
| 23|1205| prudvi|Portugal|
+---+----+-------+--------+



In [12]:
from pyspark.sql import functions as F
empleados2.groupBy(["country"])\
.agg(F.avg("age").alias("AvgAge")).show()

+--------+------+
| country|AvgAge|
+--------+------+
|Portugal|  27.6|
+--------+------+

